In [1]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
df = pd.read_csv('data/wine_data.csv')
features = df.drop('Class', axis=1)
labels = df[['Class']]

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=0)

In [4]:
Xtrain_ = torch.from_numpy(X_train.values).float()
Xtest_ = torch.from_numpy(X_test.values).float()
ytrain_ = torch.from_numpy(y_train.values).long().view(1, -1)[0]
ytest_ = torch.from_numpy(y_test.values).long().view(1, -1)[0]

In [5]:
input_size = 13
output_size = 3
hidden_size = 100

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
    

    def forward(self, X):
        X = torch.sigmoid(self.fc1(X))
        X = torch.sigmoid(self.fc2(X))
        X = self.fc3(X)

        return F.log_softmax(X, dim=-1)

In [6]:
model = Net()
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.NLLLoss()
EPOCHS = 1000
for epoch in range(EPOCHS):
    optimizer.zero_grad()
    y_pred = model(Xtrain_)
    loss = loss_fn(y_pred, ytrain_)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch: {epoch + 1}, Loss: {loss.item():.4f}')

Epoch: 100, Loss: 0.1809
Epoch: 200, Loss: 0.0500
Epoch: 300, Loss: 0.0282
Epoch: 400, Loss: 0.4018
Epoch: 500, Loss: 0.0724
Epoch: 600, Loss: 0.0480
Epoch: 700, Loss: 0.0430
Epoch: 800, Loss: 0.0407
Epoch: 900, Loss: 0.0394
Epoch: 1000, Loss: 0.0386


In [7]:
model.state_dict()

OrderedDict([('fc1.weight',
              tensor([[ 0.1120,  0.2563, -0.0408,  ..., -0.1652,  0.0526,  0.2510],
                      [-0.0658,  0.1725, -0.0683,  ..., -0.1469,  0.1052,  0.1421],
                      [ 0.2472, -0.1066, -0.0062,  ...,  0.1169, -0.0348, -0.0889],
                      ...,
                      [ 0.2008, -0.0506,  0.0597,  ..., -0.0054,  0.0826, -0.1437],
                      [-0.1649,  0.1275, -0.2037,  ...,  0.0156, -0.1733, -0.2591],
                      [-0.0364, -0.2594,  0.0709,  ...,  0.2054,  0.0909,  0.1273]])),
             ('fc1.bias',
              tensor([ 0.1240,  0.0435,  0.2725, -0.2136,  0.1732,  0.0043,  0.1174,  0.0205,
                       0.0053,  0.0290,  0.0879,  0.1010,  0.1737,  0.1239,  0.2463,  0.0033,
                      -0.0974, -0.1516, -0.2644,  0.0479, -0.2284,  0.0653,  0.2585, -0.1969,
                      -0.1416, -0.0053, -0.0455,  0.2659, -0.4787, -0.0153,  0.2852, -0.2508,
                       0.2635, -0.48

In [8]:
optimizer.state_dict()

{'state': {0: {'step': 1000,
   'exp_avg': tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
             0.0000e+00,  0.0000e+00],
           [-5.6052e-45, -5.6052e-45, -5.6052e-45,  ..., -5.6052e-45,
            -5.6052e-45, -5.6052e-45],
           [ 4.5222e-20,  9.9032e-21,  8.0828e-21,  ...,  4.0855e-21,
             1.1999e-20,  1.0817e-18],
           ...,
           [ 2.3166e-30,  5.2701e-31,  4.1767e-31,  ...,  2.1463e-31,
             6.2094e-31,  5.4833e-29],
           [-7.8384e-40, -1.8015e-40, -1.4155e-40,  ..., -7.3225e-41,
            -2.1041e-40, -1.8505e-38],
           [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
             0.0000e+00,  0.0000e+00]]),
   'exp_avg_sq': tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
            0.0000e+00],
           [2.9561e-13, 8.0763e-15, 9.4486e-15,  ..., 2.3338e-15, 1.8574e-14,
            2.0139e-10],
           [2.6139e-38, 1.2485e-39, 8.3641e-40,  ..., 2.1377e-40, 1.

In [10]:
torch.save(model.state_dict(), 'models/wine_classifier_state_dict')
!ls models

wine_classifier.pt
wine_classifier_state_dict


In [11]:
new_model = Net()

In [12]:
new_model.load_state_dict(torch.load('models/wine_classifier_state_dict'))

<All keys matched successfully>

In [13]:
new_model

Net(
  (fc1): Linear(in_features=13, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=3, bias=True)
)

In [14]:
predict_out = new_model(Xtest_)
_, predict_y = torch.max(predict_out, 1)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print(f'Accuracy: {accuracy_score(ytest_, predict_y):.4f}')
print(f'Precision: {precision_score(ytest_, predict_y, average="micro"):.4f}')
print(f'Recall: {recall_score(ytest_, predict_y, average="micro"):.4f}')
print(f'F1 Score: {f1_score(ytest_, predict_y, average="micro"):.4f}')

Accuracy: 0.9444
Precision: 0.9444
Recall: 0.9444
F1 Score: 0.9444
